In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline

import sklearn
from sklearn import linear_model
from sklearn import preprocessing

## Load dataset

In [3]:
## Load data

gss = pd.read_csv('../../GSS_2012/ICPSR_35478/DS0001/35478-0001-Data.tsv', sep='\t')
gss.head()

,YEAR,ID,INTID,FEEUSED,FEELEVEL,DATEINTV,LNGTHINV,INTAGE,INTETHN,MODE,...,SAMPCODE,SAMPLE,OVERSAMP,WTSS,WTSSNR,WTSSALL,WTCOMB,WTCOMBNR,VSTRAT,VPSU
0,2012,1,49,1,75,721,69,60,1,1,...,601,10,1,2.621963,2.869532,2.621963,6.402159,7.006659,-1,-1
1,2012,2,150,1,75,624,53,32,1,1,...,601,10,1,3.495950,3.826043,3.495950,6.514477,7.129583,-1,-1
2,2012,3,150,1,75,627,77,32,1,1,...,601,10,1,1.747975,1.913021,1.747975,1.671130,1.828920,-1,-1
3,2012,4,49,1,20,527,78,60,1,1,...,601,10,1,1.235694,1.352370,1.235694,1.181370,1.292917,-1,-1
4,2012,5,235,1,75,620,149,62,1,1,...,601,10,1,0.873988,0.956511,0.873988,0.835565,0.914460,-1,-1


## Choose variables

In [3]:
## Choose some features

gss_select = pd.DataFrame()
gss_select['id'] = gss['ID']
gss_select['wrkstat'] = gss['WRKSTAT']
## Work status

gss_select['marital'] = gss['MARITAL']
## 1 = married, 2 = widowed, 3 = divorced, 4 = separated, 5 = never married 9 = no answer

gss_select['divorce'] = gss['DIVORCE']
## 1 = yes, 2 = no, 0 = inapplicable, 8,9 = don't know/no answer

gss_select['sibs'] = gss['SIBS']
gss_select['children'] = gss['CHILDS']
gss_select['age'] = gss['AGE']
gss_select['age_1_child'] = gss['AGEKDBRN']
## Age first child born

gss_select['degree'] = gss['DEGREE']
## 0 = less than high school, 1 = high school, 2 = junior college, 
## 3 = bachelor, 4 = graduate

gss_select['sex'] = gss['SEX']
gss_select.head()

,id,wrkstat,marital,divorce,sibs,children,age,age_1_child,degree,sex
0,1,2,5,0,1,0,22,0,3,1
1,2,2,5,0,2,0,21,0,1,1
2,3,1,1,2,1,2,42,32,1,1
3,4,8,1,1,2,2,49,24,1,2
4,5,5,4,0,0,3,70,24,3,2


In [4]:
## Clean some features

gss_clean = pd.DataFrame()

## Change full time, part time and with a job (1, 2, 3) to employed, otherwise unemployed
gss_clean['wrkstat'] = np.where(gss_select['wrkstat'].isin([1, 2, 3]), 1, 0)

## Currently divorced or separated (3, 4)
gss_clean['marital'] = np.where(gss_select['marital'].isin([3, 4]), 1, 0)

## Ever divorced or separated (1)
gss_clean['divorce'] = np.where(gss_select['divorce'] == 1, 1, 0)

## 1 = Male, 2 = female, change female to 0
gss_clean['sex'] = np.where(gss_select['sex'] == 2, 0, 1)

gss_clean
gss_clean.head()

,wrkstat,marital,divorce,sex
0,1,0,0,1
1,1,0,0,1
2,1,0,0,1
3,0,0,1,0
4,0,1,0,0


In [5]:
## Create features dataframe

gss_features = gss_select.copy()
gss_features.drop(['id', 'wrkstat', 'marital', 'divorce', 'sex'], axis=1, inplace=True)
gss_features['wrkstat'] = gss_clean['wrkstat']
gss_features['sex'] = gss_clean['sex']
gss_features['div_sep'] = gss_clean['marital'] + gss_clean['divorce']
gss_features.head()

,sibs,children,age,age_1_child,degree,wrkstat,sex,div_sep
0,1,0,22,0,3,1,1,0
1,2,0,21,0,1,1,1,0
2,1,2,42,32,1,1,1,0
3,2,2,49,24,1,0,0,1
4,0,3,70,24,3,0,0,1


## Check features for sense and remove some values

In [6]:
## Check for sense - all people with no children have no 1st child age

gss_features.loc[gss_features['children'] == 0, 'age_1_child'].unique()

array([0])

In [7]:
## Remove unknown values for siblings

gss_features.sibs.unique()

array([ 1,  2,  0,  4,  6,  7,  5,  9,  3, 10, 15,  8, 13, 12, 11, 98, 99,
       14, 30, 16, 17, 22, 18, 20, 21, 58, 19])

In [8]:
sibs_drop = gss_features.loc[gss_features.sibs.isin([98, 99]),].index
gss_features.drop(sibs_drop, inplace=True)
gss_features.sibs.unique()

array([ 1,  2,  0,  4,  6,  7,  5,  9,  3, 10, 15,  8, 13, 12, 11, 14, 30,
       16, 17, 22, 18, 20, 21, 58, 19])

In [9]:
## Check number of children - 9 is don't know so remove

gss_features.children.value_counts()

2    1372
0    1250
3     784
1     696
4     390
5     155
6      75
8      52
7      35
9       4
Name: children, dtype: int64

In [10]:
children_drop = gss_features.loc[gss_features.children == 9, ].index
gss_features.drop(children_drop, inplace=True)
gss_features.children.value_counts()

2    1372
0    1250
3     784
1     696
4     390
5     155
6      75
8      52
7      35
Name: children, dtype: int64

In [11]:
## Remove no answer to age

age_drop = gss_features.loc[gss_features['age'] == 99, ].index
gss_features.drop(age_drop, inplace=True)

In [12]:
## Remove don't know and no answer to age when 1st child born

child_age_drop = gss_features.loc[gss_features['age_1_child'].isin([98, 99]),].index
gss_features.drop(child_age_drop, inplace=True)

In [13]:
## Check 0s all make sense
gss_features.loc[gss_features['age_1_child'] == 0, 'children'].unique()

array([0])

In [14]:
gss_features.degree.unique()

array([3, 1, 2, 0, 4])

In [15]:
gss_features.reset_index(inplace=True, drop=True)

In [16]:
## Data retained:

len(gss_features) / len(gss)

0.9852697095435685

In [17]:
gss_features.head()

,sibs,children,age,age_1_child,degree,wrkstat,sex,div_sep
0,1,0,22,0,3,1,1,0
1,2,0,21,0,1,1,1,0
2,1,2,42,32,1,1,1,0
3,2,2,49,24,1,0,0,1
4,0,3,70,24,3,0,0,1


## Engineer more features

In [18]:
gss_features['age_age_1_child'] = gss_features['age'] * gss_features['age_1_child']
gss_features['children_age_1_child'] = gss_features['children'] * gss_features['age_1_child']
gss_features['sibs_children'] = gss_features['sibs'] * gss_features['children']
gss_features['degree_wrkstat'] = gss_features['degree'] * gss_features['wrkstat']
gss_features['wrkstat_sex'] = gss_features['wrkstat'] * gss_features['sex']
gss_features['age_1_child_sex'] = gss_features['age_1_child'] * gss_features['sex']
gss_features['age2'] = gss_features['age'] ** 2
gss_features['age3'] = gss_features['age'] ** 3
gss_features['age_sqrt'] = gss_features['age'] ** 0.5

gss_features.head()

,sibs,children,age,age_1_child,degree,wrkstat,sex,div_sep,age_age_1_child,children_age_1_child,sibs_children,degree_wrkstat,wrkstat_sex,age_1_child_sex,age2,age3,age_sqrt
0,1,0,22,0,3,1,1,0,0,0,0,3,1,0,484,10648,4.690416
1,2,0,21,0,1,1,1,0,0,0,0,1,1,0,441,9261,4.582576
2,1,2,42,32,1,1,1,0,1344,64,2,1,1,32,1764,74088,6.480741
3,2,2,49,24,1,0,0,1,1176,48,4,0,0,0,2401,117649,7.000000
4,0,3,70,24,3,0,0,1,1680,72,0,0,0,0,4900,343000,8.366600


## Create models
### Split into train and test set

In [19]:
y_all = gss_features['div_sep']
x_all = gss_features.drop('div_sep', axis=1)

In [20]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x_all, y_all, test_size=0.3)

### Vanilla regression

In [21]:
## First attempt - set C as very large

lr_1 = linear_model.LogisticRegression(C=1e9, solver='liblinear') ## One vs rest (binary)
fit_1 = lr_1.fit(x_train, y_train)
pred_y_1 = lr_1.predict(x_train)
print(pd.crosstab(pred_y_1, y_train))
print(lr_1.score(x_train, y_train))

div_sep     0    1
row_0             
0        1894  698
1         328  404
0.6913357400722022


In [44]:
print('{0:.3f}'.format(2.344565))

2.345


In [89]:
## Use cross validation to experiment with C

def lr_cost(c_min, c_max, c_step):
    for i in np.arange(c_min, c_max, c_step):
        lr = linear_model.LogisticRegression(C=i, solver='liblinear')
        lr_score = sklearn.model_selection.cross_val_score(lr, x_train, y_train, cv=5)
        #print('lr with C = {0:0f} \nValues = {1:.3f} \nMean = {2:.3f}'.format(i, lr_score, lr_score.mean()))
        print('lr with C = ', i, ' \nValues = ', lr_score, '\nMean = ', lr_score.mean()) 

In [90]:
lr_cost(0.01, 1, 0.1)

lr with C =  0.01  
Values =  [0.6951952  0.67867868 0.67018072 0.69277108 0.69578313] 
Mean =  0.686521762726582
lr with C =  0.11  
Values =  [0.6951952  0.67117117 0.67921687 0.69126506 0.68825301] 
Mean =  0.6850202612250805
lr with C =  0.21000000000000002  
Values =  [0.69369369 0.66966967 0.67921687 0.68825301 0.6746988 ] 
Mean =  0.6811064076124318
lr with C =  0.31000000000000005  
Values =  [0.6996997  0.67117117 0.67771084 0.69126506 0.6746988 ] 
Mean =  0.6829091139332103
lr with C =  0.41000000000000003  
Values =  [0.6996997  0.66816817 0.67771084 0.68524096 0.69728916] 
Mean =  0.6856217663446579
lr with C =  0.51  
Values =  [0.69369369 0.68018018 0.67018072 0.69126506 0.69126506] 
Mean =  0.6853169434494736
lr with C =  0.6100000000000001  
Values =  [0.6996997  0.67267267 0.68072289 0.68825301 0.69277108] 
Mean =  0.6868238720648359
lr with C =  0.7100000000000001  
Values =  [0.69369369 0.68168168 0.67168675 0.69126506 0.68825301] 
Mean =  0.6853160389304966
lr with 

In [91]:
lr_cost(1, 100, 10)

lr with C =  1  
Values =  [0.6966967  0.67267267 0.67771084 0.68825301 0.68825301] 
Mean =  0.6847172473678498
lr with C =  11  
Values =  [0.6981982  0.67417417 0.67771084 0.67620482 0.68975904] 
Mean =  0.6832094142335107
lr with C =  21  
Values =  [0.69369369 0.67267267 0.67921687 0.69126506 0.68825301] 
Mean =  0.6850202612250805
lr with C =  31  
Values =  [0.69369369 0.67267267 0.67771084 0.67620482 0.6746988 ] 
Mean =  0.6789961648395383
lr with C =  41  
Values =  [0.6996997  0.67867868 0.67771084 0.68373494 0.6746988 ] 
Mean =  0.6829045913383263
lr with C =  51  
Values =  [0.6981982  0.67417417 0.67771084 0.68825301 0.68222892] 
Mean =  0.6841130286913419
lr with C =  61  
Values =  [0.6951952  0.67267267 0.67771084 0.69126506 0.68975904] 
Mean =  0.6853205615253808
lr with C =  71  
Values =  [0.69369369 0.67267267 0.67168675 0.68825301 0.69578313] 
Mean =  0.6844178515865262
lr with C =  81  
Values =  [0.69369369 0.67267267 0.67771084 0.67620482 0.68825301] 
Mean =  0.6

In [92]:
lr_cost(100, 1000, 100)

lr with C =  100  
Values =  [0.69369369 0.67867868 0.67771084 0.69427711 0.6746988 ] 
Mean =  0.6838118238720649
lr with C =  200  
Values =  [0.69369369 0.68318318 0.67771084 0.67620482 0.6746988 ] 
Mean =  0.6810982669416406
lr with C =  300  
Values =  [0.6966967  0.67267267 0.67771084 0.68524096 0.6746988 ] 
Mean =  0.6814039943558016
lr with C =  400  
Values =  [0.6996997  0.67267267 0.67771084 0.68825301 0.68674699] 
Mean =  0.6850166431491733
lr with C =  500  
Values =  [0.6996997  0.67417417 0.67771084 0.69126506 0.68975904] 
Mean =  0.686521762726582
lr with C =  600  
Values =  [0.69369369 0.68018018 0.67771084 0.67319277 0.68825301] 
Mean =  0.6826061000759795
lr with C =  700  
Values =  [0.69369369 0.67417417 0.67771084 0.69126506 0.69126506] 
Mean =  0.6856217663446579
lr with C =  800  
Values =  [0.6996997  0.67267267 0.67771084 0.68524096 0.68222892] 
Mean =  0.6835106190527878
lr with C =  900  
Values =  [0.69369369 0.67417417 0.67771084 0.69427711 0.6746988 ] 
Me

In [93]:
lr_cost(0.001, 0.1, 0.01)

lr with C =  0.001  
Values =  [0.6981982  0.67267267 0.67620482 0.69427711 0.67319277] 
Mean =  0.6829091139332103
lr with C =  0.011  
Values =  [0.6951952  0.67567568 0.68072289 0.68674699 0.68975904] 
Mean =  0.6856199573067043
lr with C =  0.020999999999999998  
Values =  [0.6996997  0.67867868 0.67771084 0.67620482 0.69427711] 
Mean =  0.6853142298925431
lr with C =  0.030999999999999996  
Values =  [0.69369369 0.67867868 0.67771084 0.68674699 0.68524096] 
Mean =  0.684414233510619
lr with C =  0.040999999999999995  
Values =  [0.69369369 0.67267267 0.67771084 0.69277108 0.69126506] 
Mean =  0.6856226708636347
lr with C =  0.05099999999999999  
Values =  [0.6996997  0.67867868 0.67771084 0.69277108 0.69427711] 
Mean =  0.6886274829045914
lr with C =  0.06099999999999999  
Values =  [0.6996997  0.67117117 0.67771084 0.68975904 0.69126506] 
Mean =  0.6859211621259814
lr with C =  0.071  
Values =  [0.69369369 0.67567568 0.67771084 0.67771084 0.69427711] 
Mean =  0.6838136329100185


In [101]:
lr_cost(1000, 10000, 1000)

lr with C =  1000  
Values =  [0.69369369 0.67567568 0.67771084 0.69427711 0.69277108] 
Mean =  0.6868256811027896
lr with C =  2000  
Values =  [0.6951952  0.67867868 0.68072289 0.68975904 0.68524096] 
Mean =  0.6859193530880278
lr with C =  3000  
Values =  [0.69369369 0.67417417 0.67771084 0.68825301 0.68975904] 
Mean =  0.6847181518868266
lr with C =  4000  
Values =  [0.69369369 0.67417417 0.67771084 0.68825301 0.6746988 ] 
Mean =  0.6817061036940555
lr with C =  5000  
Values =  [0.69369369 0.67267267 0.67771084 0.68825301 0.68975904] 
Mean =  0.6844178515865262
lr with C =  6000  
Values =  [0.69369369 0.67867868 0.67771084 0.68373494 0.6746988 ] 
Mean =  0.6817033901371251
lr with C =  7000  
Values =  [0.69369369 0.67867868 0.67771084 0.69126506 0.6746988 ] 
Mean =  0.6832094142335106
lr with C =  8000  
Values =  [0.6996997  0.67267267 0.67771084 0.69126506 0.68975904] 
Mean =  0.6862214624262817
lr with C =  9000  
Values =  [0.7012012  0.67567568 0.67771084 0.68373494 0.685

In [105]:
lr_cost(0.00001, 0.0001, 0.00001 )

lr with C =  1e-05  
Values =  [0.66666667 0.66966967 0.6626506  0.66716867 0.6686747 ] 
Mean =  0.6669660624479901
lr with C =  2e-05  
Values =  [0.66816817 0.66816817 0.66566265 0.66716867 0.6686747 ] 
Mean =  0.6675684720865444
lr with C =  3.0000000000000004e-05  
Values =  [0.66816817 0.66816817 0.66716867 0.6626506  0.67018072] 
Mean =  0.6672672672672674
lr with C =  4e-05  
Values =  [0.66816817 0.66816817 0.66566265 0.6626506  0.67168675] 
Mean =  0.6672672672672672
lr with C =  5e-05  
Values =  [0.66516517 0.66816817 0.66415663 0.65963855 0.6686747 ] 
Mean =  0.6651606425702812
lr with C =  6e-05  
Values =  [0.66666667 0.66966967 0.66114458 0.65963855 0.66716867] 
Mean =  0.6648576287130503
lr with C =  7.000000000000001e-05  
Values =  [0.66816817 0.66966967 0.65963855 0.65963855 0.6626506 ] 
Mean =  0.6639531097362422
lr with C =  8e-05  
Values =  [0.66966967 0.66816817 0.65963855 0.65963855 0.66566265] 
Mean =  0.6645555193747964
lr with C =  9e-05  
Values =  [0.66516

### Ridge regression